## Higher Order Functions

  > - Higher Order Functions are functions that operate on complex data types such as arrays and maps
  > - They allow you to pass functions as arguments (such as lambda expressions), apply transformations and return array or maps
  > - They are extremely useful for manipulating arrays without exploding them


### Commonly used Higher Order Array Functions
- TRANSFORM
- FILTER
- EXISTS
- AGGREGATE

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (array_column, lambda_expression)`
   
*lambda_expression*: `element -> expression`

In [0]:
CREATE OR REPLACE TEMPORARY VIEW order_items AS 
 SELECT * FROM VALUES
  (1, array('smartphone', 'laptop', 'monitor')),
  (2, array('tablet', 'headphones', 'smartwatch')),
  (3, array('keyboard', 'mouse'))
 AS orders(order_id, items);

In [0]:
SELECT * FROM order_items;

#### 1. Convert all the item names to be UPPERCASE (TRANSFORM Function)

In [0]:
SELECT order_id,
  TRANSFORM(items, x -> UPPER(x)) AS upper_items
  FROM order_items;

#### 2. Filter only items that contains the string 'smart' (FILTER Function)

In [0]:
SELECT order_id,
  FILTER(items, x -> x LIKE '%smart%') AS smart_items
  FROM order_items;

#### Check to see whether the order includes any 'monitor' (EXISTS Function)

In [0]:
SELECT order_id,
  EXISTS(items, x -> x = 'monitor') AS has_monitor
  FROM order_items;

### Array with more than one object

In [0]:
CREATE OR REPLACE TEMPORARY VIEW order_items AS 
 SELECT * FROM VALUES
  (1, array(
    named_struct('name', 'smartphone', 'price', 699), 
    named_struct('name', 'laptop', 'price', 1199),
    named_struct('name', 'monitor', 'price', 399)
    )),
  (2, array(
    named_struct('name', 'tablet', 'price', 599), 
    named_struct('name', 'headphones', 'price', 199), 
    named_struct('name', 'smartwatch', 'price', 299)
    )),
  (3, array(
    named_struct('name', 'keyboard', 'price', 89), 
    named_struct('name', 'mouse', 'price', 59)
    ))
 AS orders(order_id, items);

In [0]:
SELECT * FROM order_items;

#### 1. Convert all the items names to be UPPERCASE & Add 10% TAX to each item (TRANSFORM Function)

In [0]:
SELECT order_id,
  TRANSFORM(items, x -> named_struct(
                                      'name', UPPER(x.name),
                                      'price', ROUND(x.price * 1.10, 2))) AS item_name
  FROM order_items;

#### 2. Calculate the total order amount for each of the orders (AGGREGATE Function)

In [0]:
SELECT order_id,
  AGGREGATE(items, 0, (acc, x) -> acc + x.price) AS total_amount
  FROM order_items;

## Map Functions

A map is a collection of key-value pairs, like a dictionary

`{'laptop': 1200, 'phone': 699}`


### Commonly used Higher Order Map Functions
- TRANSFORM_VALUES
- TRANSFORM_KEYS
- MAP_FILTER

#### Syntax
-----------------------------------------------------------------------------
`<function_name> (map_column, lambda_expression)`
   
*lambda_expression*: `(key, value) -> expression`

In [0]:
CREATE OR REPLACE TEMPORARY VIEW order_item_prices AS 
 SELECT * FROM VALUES
  (1, map('smartphone', 699, 'laptop', 1199, 'monitor', 399)),
  (2, map('tablet', 599, 'headphones', 199, 'smartwatch', 299)),
  (3, map('keyboard', 89, 'mouse', 59))
 AS orders(order_id, item_prices);

In [0]:
SELECT * FROM order_item_prices

#### 1. Convert all the item names to be UPPERCASE (TRANSFORM_KEYS Function)

In [0]:
SELECT order_id,
       TRANSFORM_KEYS(item_prices, (k, v) -> UPPER(k)) AS items_upper_case
 FROM order_item_prices

#### 2. Apply 10% TAX to item prices (TRANSFORM_VALUES Function)

In [0]:
SELECT order_id,
       TRANSFORM_VALUES(item_prices, (k, v) -> ROUND(v * 1.10, 2)) AS prices_with_tax
 FROM order_item_prices

#### 3. Filter only items with price above $500 (MAP FILTER Function)

In [0]:
SELECT order_id,
       MAP_FILTER(item_prices, (k, v) -> v > 500) AS items_above_500
 FROM order_item_prices